## Let's first import the necessary libraries.

In [44]:
#!pip install alpaca-py
#!pip install scikit-learn
from alpaca.trading.requests import MarketOrderRequest 
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.client import TradingClient
trading_client = TradingClient('PKZ2QZLDNOZ3NAH5BBSC', 'zZOheM14bQ35PJkCHrwyU4CgMcSke6sFvdbWPD7S')
print(trading_client.get_account().account_number)
print(trading_client.get_account().buying_power)
from alpaca.data import StockHistoricalDataClient, StockTradesRequest, StockBarsRequest
from datetime import datetime
from datetime import timedelta
from alpaca.data import CryptoHistoricalDataClient,CryptoTradesRequest, CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split



data_client = StockHistoricalDataClient('PKZ2QZLDNOZ3NAH5BBSC', 'zZOheM14bQ35PJkCHrwyU4CgMcSke6sFvdbWPD7S')
FXdata_client = CryptoHistoricalDataClient('PKZ2QZLDNOZ3NAH5BBSC', 'zZOheM14bQ35PJkCHrwyU4CgMcSke6sFvdbWPD7S')
trading_client = TradingClient('PKZ2QZLDNOZ3NAH5BBSC', 'zZOheM14bQ35PJkCHrwyU4CgMcSke6sFvdbWPD7S')
# market_order_data = MarketOrderRequest(
#     symbol = 'SOL', #Good ETF to buy for introduction
#     qty = 3,
#     side = OrderSide.BUY,
#     time_in_force = TimeInForce.DAY #order's opened till EOD or cancel
#     #This is a market order request, there's no limit price that we want to buy at, we're just buying no matter
#     # price SPY's currently at.

# )
# market_order = trading_client.submit_order(market_order_data)

def makeOrder(symbol, numShares, orderType, type):
    tforce = None
    if type == 'stock':
        tforce = TimeInForce.DAY 
    else:
        tforce = TimeInForce.GTC 
    oside = None
    if orderType == 'buy':
        oside = OrderSide.BUY
    else:
        oside = OrderSide.SELL

    market_order_data = MarketOrderRequest(
        symbol = symbol, #Good ETF to buy for introduction
        qty = numShares,
        side = oside,
        time_in_force = tforce
        )
    market_order = trading_client.submit_order(market_order_data)
    
    #order's opened till EOD or cancel
        #This is a market order request, there's no limit price that we want to buy at, we're just buying no matter
        # price SPY's currently at.

def portfolioSize(symbol):
    try:
        open_positions = trading_client.get_open_position(symbol)
        qty = open_positions.qty_available
        return qty
    #     print(len(open_positions))
    except:
        return 0



# #print(open_positions)
# #print(len(trading_client.get_all_positions()))


PA37F7DIWXPQ
199457.98


## I'd now like to develop a function that will allow me to pull the most recent bars for a set amount of time from now in dataframe form in order to develop my linear regression. Given that I plan to potentially work with different types of data, I'd like to be able to pull both Stock and Cryptocurrency data.

In [42]:
import time
def currentTrades(symbol, minutesBack, type):
    print(symbol)
    currentDate = datetime.now()
    delta = timedelta(minutes= - minutesBack)
    previousDate = currentDate + delta
    if type == 'stock':
        request_params = StockBarsRequest(symbol_or_symbols = symbol, timeframe = TimeFrame.Minute, start = previousDate, end = currentDate)
        bars = data_client.get_stock_bars(request_params)
        df = bars.df
        return df
    else:
        request_params = CryptoBarsRequest(symbol_or_symbols = symbol, timeframe = TimeFrame.Minute, start = previousDate, end = currentDate)
        bars = FXdata_client.get_crypto_bars(request_params)
        df = bars.df
        return df        


#df = periodic_work(60, 'AAPL', 60)

## Now, I'd like to try and develop a model for a Linear Regression based on the Open, High, Low, and Close WVAP as predictors and the Close as the target, using the signal as described in the README file.

In [ ]:
def executeStrategy(interval, symbol, minutesBack, type):
    while True:
        df = currentTrades(symbol, minutesBack, type)
       # (((high * low)^0.5) - vwap)
        print(len(df))
        df['signal'] = ((np.power(0.5, (df['high'] * df['low']))) - df['vwap'])
        df['next_close'] = df['close'].shift(-1).dropna()
        recentSignal = df['signal'][-1]
        print(recentSignal)
        finalClose = df['close'][-1]
        df = df.drop(columns = ['volume', 'trade_count', 'open', 'high', 'low','vwap' ])
        df = df.dropna()
        #print(df[['close', 'next_close', 'signal']])
        print(df.columns)

        #Now, let's create our model.
        model = LinearRegression()
        X = df['signal']
        y = df['next_close']
        print(len(X))
        print(len(y))
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 23)
        X_train = np.array(X_train).reshape(-1, 1)
        print(len(X_train))
        print(len(y_train))
        model.fit(X_train, y_train)
        coefficient = model.coef_[0]
        intercept = model.intercept_
        print(f'Equation:  y = {coefficient}x + {intercept}')



        predictedPrice =  coefficient*recentSignal + intercept #just using the actual parameters here instead of predict.clear
        print(finalClose)
        print(predictedPrice)
        if predictedPrice > finalClose:
            if portfolioSize(symbol) < 2:
                makeOrder(symbol, 1, 'buy', type)
                print("BOUGHT SHARE")
            else:
                print("DID NOTHING")
            #want to buy! however, for the time being I want to keep my portfolio small, so I will only allow us to have bought 2 shares 
        else:
            if portfolioSize(symbol) > 0:
                makeOrder(symbol, portfolioSize(symbol), 'sell', type)
                print("SOLD SHARE")
            else:
                print("DID NOTHING")
            #want to sell here
        

       # df.plot(x='signal', y='next_close', kind='scatter')
        time.sleep(interval)
        
executeStrategy(3600, 'SOL/USD', 100, 'crypto')

SOL/USD
92
-172.92275
Index(['close', 'signal', 'next_close'], dtype='object')
91
91
72
72
Equation:  y = -0.8381880671743012x + 27.930954883666885
172.905
172.87274047663178


/var/folders/3s/gp2ztt_d2cn_dn33vj1cbx2h0000gn/T/ipykernel_1225/3683373354.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  recentSignal = df['signal'][-1]
/var/folders/3s/gp2ztt_d2cn_dn33vj1cbx2h0000gn/T/ipykernel_1225/3683373354.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  finalClose = df['close'][-1]


DID NOTHING
